In [ ]:
# Imports
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, Input, Output
import numpy as np
import dash_bootstrap_components as dbc
import dash_daq as daq
from simulation.functions import runSimulation, aggregateOutputs, readData
import os
import sys

# Datasets
datasets = readData(area="kamppi")
dataset_names = list(datasets.keys())
max_sim_timestep = datasets["Current emissions"]["Simulation timestep"].max()/60
timeline = np.arange(1, max_sim_timestep+1, 1)
####-----------------------------------------

# Global variables
mobility_modes = ["Pedestrians", "Private cars", "Bicycles", "Public transport"]
objectives = ["Summary", "Air quality", "Livability", "Traffic"]
seasons = ["Winter", "Summer"]
weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
mobility_demand = ["Low", "Realistic", "High"]
areas = ["Kamppi"]

####-----------------------------------------

# Traffic variables
traffic_variables = ["Moblility flows",
                     "Trips",
                     "Average speed",
                     "Travel time",
                     "Lost time",
                     "Noise"]
traffic_cols = {"Mobility flows": "Amount", 
                "Trips": "Mobility mode",
                "Average speed": "Speed",
                "Travel time": "Travel time",
                "Lost time": "Lost time",
                "Noise": "Noise"}
traffic_units = {"Mobility flows": "# of vehicles",
                "Trips": "# of trips",
                "Average speed": "m/s",
                "Travel time": "seconds",
                "Lost time": "seconds",
                "Noise": "dB"}

####-----------------------------------------

# AQ variables
pm25_thresholds = [0, 10, 25, 50, 75, np.inf]
bc_thresholds = [0, 1, 3, 7, 12, np.inf]
ntot_thresholds = [0, 15, 30, 60, 100, np.inf] 
aqi_thresholds = {0: "good", 1: "satisfactory", 2: "fair", 3: "poor", 4: "very poor"}
aq_variables = [
    # "Air quality",
                # "Black carbon",
                "Carbon monoxide",
                "Carbon dioxide",
                "Hydrocarbon",
                "Nitrogen oxides",
                "Respirable particles"]
# Fine particles
# Ntot is a sum of fine and coarse particles
aq_units = {
# "Air quality": "",
# "Black carbon": "µg", 
"Carbon monoxide": "mg",
"Carbon dioxide": "mg",
"Hydrocarbon": "mg",
"Nitrogen oxides": "mg",
"Respirable particles": "µg"
# "Finer particles": "µg"
}
# aq_maxes = 2*datasets["Current emissions"][aq_variables].max(axis=0)
# aq_maxes = aq_maxes.to_dict()

####-----------------------------------------

# Livability variables

####-----------------------------------------

# Helper function to normalize to an array to a range (x,y)
def normalize_range(array, x, y):
    m = min(array)
    range = max(array) - m
    array = (array - m)/range
    range2 = y - x
    normalized = (array*range2) + x
    return list(normalized)

def filter_dataframe(dataset_name="Current emissions",
                     spatial_click_data=None,
                     timestep_range=1,
                     timeline_type="Average",
                     variable="Carbon dioxide",
                     network_bounds=True):
    data = datasets[dataset_name]
    # Make a copy of the required data (to avoid changing the simulation output)
    required_cols = ["Name", "Edge", "Simulation timestep", "Longitude", "Latitude", "Amount"]
    if variable != "Amount":
        required_cols.append(variable)
    if variable != "Noise":
        required_cols.append("Mobility mode")
    if "trip" in dataset_name:
        required_cols = ["Trip","Travel time","Lost time","Mobility mode"] 
    network = data[required_cols].copy()
    map_bounds = {}
    if network_bounds:
        # Set the bounds for heatmap based on the whole data
        west_bound = network["Longitude"].min()
        east_bound = network["Longitude"].max()
        south_bound = network["Latitude"].min()
        north_bound = network["Latitude"].max()
        map_bounds = {
            "west": west_bound * (1 - 2e-4),
            "east": east_bound * (1 + 2e-4),
            "south": south_bound * (1 - 5e-6),
            "north": north_bound * (1 + 5e-6),
        }
    # Set the location text and spatial scope for the data when the location is changed
    if spatial_click_data is None:
        location = "Location: Network"
    else:
        road_lon = spatial_click_data["points"][0]["lon"]
        road_lat = spatial_click_data["points"][0]["lat"]
        name = spatial_click_data["points"][0]["customdata"][0]
        location = f"Location: {name} ({road_lat:.2f} °N, {road_lon:.2f} °E)"
        edge = spatial_click_data["points"][0]["customdata"][1]
        network = network[network["Edge"] == edge]
    # Re-index timesteps from simulation seconds to minutes
    timesteps = np.arange(network["Simulation timestep"].min(), network["Simulation timestep"].max(), timestep_range*60, dtype=int)
    timestep_labels = timesteps//60
    network["Timestep"] = pd.cut(network["Simulation timestep"], 
                                bins=timesteps,
                                labels=timestep_labels[1:],
                                right=False, 
                                include_lowest=True)
    # Re-calculate the data when the aggregation method is changed
    if timeline_type == "Average":
        func = "mean"
    else:
        func = "sum"
    grouping_cols = ["Timestep", "Edge"]
    if "noise" not in dataset_name:
        grouping_cols.append("Mobility mode")
    network = network.groupby(grouping_cols, observed=False).agg(
        {variable: func, "Longitude": "first", "Latitude": "first", "Name": "first", "Amount": func}
        ).reset_index()
    return network, location, map_bounds

####-----------------------------------------

# App

# Style
external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.BOOTSTRAP]

# Hover layout
hovers=dict(bgcolor="white", font_size=16)

# Main app
app = Dash(__name__,
           external_stylesheets=external_stylesheets,
           suppress_callback_exceptions=True)

# Main layout
app.layout = html.Div(
    [
        # Scenario params
        html.Div(
            [
                # Title
                html.H2("Scenario parameters", style={"display": "inline-block"}),
                dbc.Row(
                    [
                        dbc.Col(
                            # Buttons for season
                            html.Div(
                                [
                                    html.Label(
                                        "Season",
                                        style={
                                            "font-size": 24,
                                            "padding-bottom": "2vh",
                                            "padding-top": "1vh",
                                        },
                                    ),
                                    dcc.Dropdown(
                                        options=seasons,
                                        value="Summer",
                                        id="crossfilter-season",
                                        style={
                                            "font-size": 18,
                                            "width": "95%",
                                        },
                                    ),
                                ],
                                style={"padding-top": "2vh", "padding-bottom": "2vh"},
                            ),
                        ),
                        dbc.Col(
                            # Dropdown for weekday
                            html.Div(
                                [
                                    html.Label(
                                        "Weekday",
                                        style={
                                            "font-size": 24,
                                            "padding-bottom": "2vh",
                                            "padding-top": "1vh",
                                        },
                                    ),
                                    dcc.Dropdown(
                                        options=weekdays,
                                        value="Mon",
                                        id="crossfilter-day",
                                        style={
                                            "font-size": 18,
                                            "width": "95%",
                                        },
                                    ),
                                ],
                                style={"padding-top": "2vh", "padding-bottom": "2vh"},
                            ),
                        ),
                        dbc.Col(
                            # Buttons for traffic demand
                            html.Div(
                                [
                                    html.Label(
                                        "Demand",
                                        style={
                                            "font-size": 24,
                                            "padding-bottom": "2vh",
                                            "padding-top": "1vh",
                                        },
                                    ),
                                    dcc.Dropdown(
                                        options=mobility_demand,
                                        value="Realistic",
                                        id="crossfilter-mobility-demand",
                                        style={
                                            "font-size": 18,
                                            "width": "95%",
                                        },
                                    ),
                                ],
                                style={"padding-top": "2vh", "padding-bottom": "2vh"},
                            ),
                        ),
                        dbc.Col(
                            # Dropdown for area
                            html.Div(
                                [
                                    html.Label(
                                        "Area",
                                        style={
                                            "font-size": 24,
                                            "padding-bottom": "2vh",
                                            "padding-top": "1vh",
                                        },
                                    ),
                                    dcc.Dropdown(
                                        options=areas,
                                        value="Kamppi",
                                        id="crossfilter-area",
                                        style={
                                            "font-size": 18,
                                            "width": "95%",
                                        },
                                    ),
                                ],
                                style={"padding-top": "2vh", "padding-bottom": "2vh"},
                            ),
                        ),
                    ]
                ),
            ],
        ),
        # Dividing line between the parameters
        html.Hr(),
        # Title
        html.H2(
            "Optimization parameters",
            style={"padding-top": "2vh", "padding-bottom": "2vh"},
        ),
        # Optimization params
        html.Div(
            [
                dbc.Row(
                    [
                        dbc.Col(
                            html.Label(
                                "Objective",
                            ),
                            width={"size": 2},
                        ),
                        dbc.Col(
                            html.Center(
                                html.Label(
                                    "Priority",
                                ),
                            ),
                        ),
                        dbc.Col(
                            html.Center(
                                html.Label(
                                    "Performance thresholds",
                                ),
                            ),
                        ),
                    ],
                    style={
                        "padding-bottom": "2vh",
                        "padding-top": "1vh",
                        "font-size": 24,
                    },
                ),
                dbc.Row(
                    [
                        dbc.Col(
                            html.Label(
                                "Traffic",
                                style={
                                    "font-size": 18,
                                },
                            ),
                            width={"size": 2},
                        ),
                        dbc.Col(
                            dcc.Slider(
                                min=0,
                                max=100,
                                step=10,
                                value=50,
                                id="traffic-optim-slider",
                            ), align="end",
                        ),
                        dbc.Col(
                            html.Center(
                                html.Div(
                                    [
                                        daq.NumericInput(
                                            value=100,
                                            id="crossfilter-traffic-optim-threshold",
                                            min=0,
                                            max=100,
                                            label="% of improvement",
                                            labelPosition="bottom",
                                            style={
                                                "display": "inline-block",
                                                "font-size": 18,
                                                "padding-right": "1vw",
                                            },
                                        ),
                                        # html.P(
                                        #     f"%",
                                        #     style={
                                        #         "display": "inline-block",
                                        #         "font-size": 18,
                                        #     },
                                        # ),
                                    ]
                                ),
                            ),
                        ),
                    ],
                    style={
                        "font-size": 18,
                        "padding-bottom": "2vh",
                        "padding-top": "2vh",
                    },
                    align="center",
                ),
                dbc.Row(
                    [
                        dbc.Col(
                            html.Label(
                                "Air quality",
                                style={
                                    "font-size": 18,
                                },
                            ),
                            width={"size": 2},
                        ),
                        dbc.Col(
                            dcc.Slider(
                                min=0,
                                max=100,
                                step=10,
                                value=50,
                                id="air-quality-optim-slider",
                            ), align="end",
                        ),
                        dbc.Col(
                            html.Center(
                                html.Div(
                                    [
                                        daq.NumericInput(
                                            value=100,
                                            id="crossfilter-aq-optim-threshold",
                                            min=0,
                                            max=100,
                                            label="% of improvement",
                                            labelPosition="bottom",
                                            style={
                                                "display": "inline-block",
                                                "font-size": 18,
                                                "padding-right": "1vw",
                                            },
                                        ),
                                        # html.P(
                                        #     f"%",
                                        #     style={
                                        #         "display": "inline-block",
                                        #         "font-size": 18,
                                        #     },
                                        # ),
                                    ]
                                ),
                            ),
                        ),
                    ],
                    style={
                        "font-size": 18,
                        "padding-bottom": "2vh",
                        "padding-top": "2vh",
                    },
                    align="center",
                ),
                dbc.Row(
                    [
                        dbc.Col(
                            html.Label(
                                "Livability",
                                style={
                                    "font-size": 18,
                                },
                            ),
                            width={"size": 2},
                        ),
                        dbc.Col(
                            dcc.Slider(
                                min=0,
                                max=100,
                                step=10,
                                value=50,
                                id="livability-optim-slider",
                            ), align="end",
                        ),
                        dbc.Col(
                            html.Center(
                                html.Div(
                                    [
                                        daq.NumericInput(
                                            value=100,
                                            id="crossfilter-liv-optim-threshold",
                                            label="% of improvement",
                                            labelPosition="bottom",
                                            min=0,
                                            max=100,
                                            style={
                                                "display": "inline-block",
                                                "font-size": 18,
                                                "padding-right": "1vw",
                                            },
                                        ),
                                        # html.P(
                                        #     f"%",
                                        #     style={
                                        #         "display": "inline-block",
                                        #         "font-size": 18,
                                        #     },
                                        # ),
                                    ]
                                ),
                            ),
                        ),
                    ],
                    style={
                        "font-size": 18,
                        "padding-bottom": "1vh",
                        "padding-top": "1vh",
                    },
                    align="center",
                ),
            ]
        ),
        # Simulate button
        html.Div(
            html.Center(
                dbc.Button(
                    html.Label(
                        "Simulate",
                        style={
                            "font-size": 18,
                        },
                    ),
                    size="lg",
                ),
                style={
                    "padding-top": "2vh",
                    "padding-bottom": "2vh",
                    "display": "block",
                },
            ), style={"padding-top": "3vh"}
        ),
        # Dividing line between the parameters
        html.Hr(),
        # Viz params
        html.Div(
            [
                # Title
                html.H2(
                    "Visualization parameters",
                    style={"padding-top": "2vh", "padding-bottom": "2vh"},
                ),
            ]
        ),
        # Toggles and texts for parameters
        dbc.Row(
            [
                # Timeline type column
                dbc.Col(
                    html.Div(
                        [
                            html.Label(
                                "Temporal aggregation", style={"padding-bottom": "2vh"}
                            ),
                            dcc.RadioItems(
                                options=["Sum", "Average"],
                                value="Sum",
                                id="crossfilter-timeline-type",
                                inline=True,
                                labelStyle={
                                    "automargin": True,
                                    "padding-top": "1vh",
                                    "font-size": 18,
                                    "padding-right": "1vw",
                                },
                                inputStyle={"margin-right": "0.3vw"},
                            ),
                        ]
                    )
                ),
                # Timestep interval column
                dbc.Col(
                    html.Div(
                        [
                            html.Label(
                                "Timestep interval", style={"padding-bottom": "2vh"}
                            ),
                            html.Div(
                                [
                                    daq.NumericInput(
                                        value=1,
                                        id="crossfilter-timestep-range",
                                        label="minute(s)",
                                        labelPosition="bottom",
                                        min=1,
                                        max=timeline.max(),
                                        style={
                                            "display": "inline-block",
                                            "font-size": 18,
                                            "padding-right": "1vw",
                                        },
                                    ),
                                    # html.P(
                                    #     "minute(s)",
                                    #     style={
                                    #         "display": "inline-block",
                                    #         "font-size": 18,
                                    #     },
                                    # ),
                                ]
                            ),
                        ]
                    )
                ),
                # Row end
            ],
            justify="center",
            style={"automargin": True, "padding-top": "2vh", "font-size": 24},
        ),
        # Dividing line between the params and plots
        html.Hr(),
        # Title
        html.H2("Results", style={"padding-bottom": "2vh", "padding-top": "1vh"}),
        # Tabs
        dcc.Tabs(
            id="tabs-1",
            value="air-quality",
            children=[
                dcc.Tab(label="Summary", value="summary"),
                dcc.Tab(label="Traffic", value="traffic"),
                dcc.Tab(label="Air quality", value="air-quality"),
                dcc.Tab(label="Livability", value="livability"),
            ],
        ),
        html.Div(id="tabs-content-1", style={"padding-top": "1vh"}),
        # Main layout style
    ],
    style={"width": "99vw", "automargin": True, "padding": "5vh 5vh 5vh 5vh"},
)

####-----------------------------------------

# Callbacks

# Tab update callback
@app.callback(
    Output('tabs-content-1', 'children'),
    Input('tabs-1', 'value')
)
def renderViewContent(tab):

    ####-----------------------------------------
    # View for summary tab
    if tab == 'summary':
        return html.Label("Summary under work", style={"padding-top": "2vh", "font-size": 20})

    ####-----------------------------------------
    # View for traffic tab
    elif tab == 'traffic':
        return html.Center(
            [
                # Current or optimized situation
                html.Div(
                    [
                        html.Label(
                            "Situation",
                            style={
                                "font-size": 24,
                                "padding-bottom": "2vh",
                                "padding-top": "1vh",
                            },
                        ),
                        dcc.RadioItems(
                            options=["Current", "Optimized"],
                            value="Current",
                            id="crossfilter-situation-traffic",
                            inline=True,
                            labelStyle={
                                "automargin": True,
                                "padding-top": "1vh",
                                "font-size": 18,
                                "padding-right": "1vw",
                            },
                            inputStyle={"margin-right": "0.4vw"},
                        ),
                    ],
                    style={"padding-top": "2vh", "padding-bottom": "2vh"},
                ),
                # Variable dropdown
                html.Div(
                    [
                        html.Div(
                            [
                                html.Label(
                                    "Variable",
                                    style={
                                        "font-size": 24,
                                        "padding-bottom": "2vh",
                                        "padding-top": "1vh",
                                    },
                                ),
                                dcc.Dropdown(
                                    options=traffic_variables,
                                    value="Traffic flow",
                                    id="crossfilter-variable-traffic",
                                    style={
                                        "font-size": 18,
                                        "width": "50%",
                                        "margin": "auto",
                                        "text-align": "center",
                                    },
                                ),
                            ],
                            style={"padding-top": "2vh", "padding-bottom": "2vh"},
                        ),
                        # Network plots
                        html.Div(
                            [
                                # The heatmap
                                dcc.Loading(
                                    dcc.Graph(
                                        id="traffic-heatmap",
                                        responsive=True,
                                        style={"height": "45vw",
                                               "width": "68vw",},
                                        config={"showTips": True},
                                    ),
                                    type="cube",
                                ),
                            ],
                            style={
                                "padding-top": "3vh",
                            },
                        ),
                        # Drill-down plots
                        html.Div(
                            [
                                # Title
                                html.Label(
                                    html.Pre(id="location-text-traffic"),
                                    style={
                                        "font-size": 24,
                                        "padding-bottom": "4vh",
                                        "display": "block",
                                        "padding-top": "2vh",
                                    },
                                ),
                                # Reset button
                                dbc.Button(
                                    "Reset location",
                                    id="reset-button-traffic",
                                    n_clicks=0,
                                    style={
                                        "display": "block",
                                        "font-size": 18,
                                        "automargin": True,
                                        "padding": "1vh",
                                    },
                                ),
                                # Bar plot
                                # dcc.Loading(
                                #     dcc.Graph(
                                #         id="traffic-bar",
                                #         responsive=True,
                                #         style={"height": "30vw"},
                                #     ),
                                #     type="cube",
                                # ),
                                # Stream graph
                                dcc.Loading(
                                    dcc.Graph(
                                        id="traffic-area",
                                        responsive=True,
                                        style={"height": "30vw"},
                                    ),
                                    type="cube",
                                ),
                            ],
                            style={"padding-top": "3vh"},
                        ),
                    ]
                ),
            ]
        )

    ####-----------------------------------------
    # View for AQ tab
    elif tab == 'air-quality':
        return html.Center(
            [
                # Current or optimized situation
                html.Div(
                    [
                        html.Label(
                            "Situation",
                            style={
                                "font-size": 24,
                                "padding-bottom": "2vh",
                                "padding-top": "1vh",
                            },
                        ),
                        dcc.RadioItems(
                            options=["Current", "Optimized"],
                            value="Current",
                            id="crossfilter-situation-aq",
                            inline=True,
                            labelStyle={
                                "automargin": True,
                                "padding-top": "1vh",
                                "font-size": 18,
                                "padding-right": "1vw",
                            },
                            inputStyle={"margin-right": "0.4vw"},
                        ),
                    ],
                    style={"padding-top": "2vh", "padding-bottom": "2vh"},
                ),
                # Variable dropdown
                html.Div(
                    [
                        html.Div(
                            [
                                html.Label(
                                    "Variable",
                                    style={
                                        "font-size": 24,
                                        "padding-bottom": "2vh",
                                        "padding-top": "1vh",
                                    },
                                ),
                                dcc.Dropdown(
                                    options=aq_variables,
                                    value="Carbon monoxide",
                                    id="crossfilter-variable-aq",
                                    style={
                                        "font-size": 18,
                                        "width": "50%",
                                        "margin": "auto",
                                        "text-align": "center",
                                    },
                                ),
                            ],
                            style={"padding-top": "2vh", "padding-bottom": "2vh"},
                        ),
                        # Network plots
                        html.Div(
                            [
                                # The heatmap
                                dcc.Loading(
                                    dcc.Graph(
                                        id="aq-heatmap",
                                        responsive=True,
                                        config={"showTips": True},
                                        style={"height": "45vw",
                                               "width": "68vw"}
                                    ),
                                    type="cube",
                                ),
                            ],
                            style={"padding-top": "3vh"},
                        ),
                        # Drill-down plots
                        html.Div(
                            [
                                # Title
                                html.Label(
                                    html.Pre(id="location-text-aq"),
                                    style={
                                        "font-size": 24,
                                        "padding-bottom": "4vh",
                                        "display": "block",
                                        "padding-top": "2vh",
                                    },
                                ),
                                # Reset button
                                dbc.Button(
                                    "Reset location",
                                    id="reset-button-aq",
                                    n_clicks=0,
                                    style={
                                        "display": "block",
                                        "font-size": 18,
                                        "automargin": True,
                                        "padding": "1vh",
                                    },
                                ),
                                # Bar plot
                                dcc.Loading(
                                    dcc.Graph(
                                        id="aq-bar",
                                        responsive=True,
                                        config={"showTips": True},
                                        style={"height": "30vw"},
                                    ),
                                    type="cube",
                                ),
                                # Stream graph
                                dcc.Loading(
                                    dcc.Graph(
                                        id="aq-area",
                                        responsive=True,
                                        style={"height": "30vw"},
                                    ),
                                    type="cube",
                                ),
                            ],
                            style={"padding-top": "3vh"},
                        ),
                    ]
                ),
            ]
        )

    ####-----------------------------------------
    # View for livability tab
    elif tab == 'livability':

        return html.Center([

        # Current or optimized situation
        html.Div([
            html.Label("Situation", style={"font-size": 24, "padding-bottom": "2vh", "padding-top": "1vh"}),
            dcc.RadioItems(options=["Current", "Optimized"],
                value="Current",
                id="crossfilter-situation-liv",
                inline=True, 
                labelStyle={"automargin": True, "padding-top": "1vh", "font-size": 18, "padding-right": "1vw"}, 
                inputStyle={"margin-right": "0.4vw"}),                
            ],
            style={"padding-top": "2vh", "padding-bottom": "2vh"}
        ),
            
        html.Label("Livability under work", style={"padding-top": "2vh", "font-size": 20}),

        ])


####-----------------------------------------

## Traffic

# Callback to traffic heatmap
@app.callback(
    Output("traffic-heatmap", "figure"),
    Output("location-text-traffic", "children"),
    Input("crossfilter-situation-traffic", "value"), 
    Input("crossfilter-variable-traffic", "value"),
    Input("crossfilter-timestep-range", "value"),
    Input("crossfilter-timeline-type", "value"),
    Input("traffic-heatmap", "clickData"))
def drawTrafficHeatmap(situation, variable, timestep_range, timeline_type, spatial_click_data):
    # Which dataset to use
    dataset_name = ""
    if variable == "Traffic flow":
        if situation == "Current":
            dataset_name = "Current emissions"
        else:
            dataset_name = "Optimized emissions"
    elif variable == "Noise":
        if situation == "Current":
            dataset_name = "Current edge noise"
        else:
            dataset_name = "Optimized edge noise"
    elif variable == "Average speed":
        if situation == "Current":
            dataset_name = "Current trips"
        else:
            dataset_name = "Optimized trips"
    # Map the traffic variable to the traffic column
    traffic_column = traffic_cols[variable]
    # Filter the dataset according to inputs
    network, location, bounds = filter_dataframe(dataset_name=dataset_name,
    spatial_click_data=spatial_click_data,
    timestep_range=timestep_range,
    timeline_type=timeline_type,
    variable=traffic_column)
    # Draw the figure
    legend_max = 1.2 * network[network[traffic_column] != 0][variable].median()
    fig = px.density_map(
        data_frame=network,
        lat="Latitude",
        lon="Longitude",
        z=traffic_column,
        radius=15,
        map_style="open-street-map",
        range_color=(0, legend_max),
        hover_data={
            "Name": True,
            "Edge": False,
            "Amount": True,
            "Longitude": False,
            "Latitude": False,
            traffic_column: True,
        },
        labels={traffic_column: variable},
        animation_frame="Timestep",
        title="Network heatmap",
    )
    # Customize the sliders with a larger font size and prefix, retain the active frame
    sliders = [dict(
    currentvalue={"prefix": "Time (in minutes): "},
    font={"size": 14},
    pad={"t": 50}
    )]
    # Update points' opacity on click
    fig.update_layout(
        clickmode="event+select",
        sliders=sliders,
        map_bounds=bounds
    )
    # Sets the variable name and its unit as a legend to the color bar
    fig.layout["coloraxis"]["colorbar"]["title"] = f"{variable} ({traffic_units[variable]})"
    # Return the figure and location
    return fig, location

# Callback to traffic stacked area chart
@app.callback(
    Output("traffic-area", "figure"),
    Input("crossfilter-situation-traffic", "value"),
    Input("crossfilter-variable-traffic", "value"),
    Input("crossfilter-timestep-range", "value"),
    Input("crossfilter-timeline-type", "value"),
    Input("traffic-heatmap", "clickData"))
def drawTrafficArea(situation, variable, timestep_range, timeline_type, spatial_click_data):
    # Which dataset to use
    dataset_name = ""
    if variable == "Traffic flow":
        if situation == "Current":
            dataset_name = "Current emissions"
        else:
            dataset_name = "Optimized emissions"
    elif variable == "Noise":
        if situation == "Current":
            dataset_name = "Current edge noise"
        else:
            dataset_name = "Optimized edge noise"
    elif variable == "Average speed":
        if situation == "Current":
            dataset_name = "Current trips"
        else:
            dataset_name = "Optimized trips"
    # Map the traffic variable to the traffic column
    traffic_column = traffic_cols[variable]
    # Filter the dataset according to inputs
    network, location, bounds = filter_dataframe(
    dataset_name=dataset_name,
    spatial_click_data=spatial_click_data,
    timestep_range=timestep_range,
    timeline_type=timeline_type,
    variable=traffic_column)
    # Calculate mobility mode values
    if traffic_column != "Amount":
        network = network.groupby(["Mobility mode", "Timestep"], observed=False).agg(
            {traffic_column: "sum", "Amount": "sum"}
            ).reset_index()
        # Draw the figure
        area_plot = px.area(
            data_frame=network, 
            x="Timestep", 
            y=traffic_column, 
            hover_data="Amount",
            color="Mobility mode", 
            color_discrete_sequence=px.colors.sequential.Plasma_r,
            labels={traffic_column: variable},
            title="Mobility mode time series"
        )
    else:
        network = network.groupby(["Mobility mode", "Timestep"], observed=False).agg(
            {traffic_column: "sum"}
            ).reset_index()
        # Draw the figure
        area_plot = px.area(
            data_frame=network,
            x="Timestep",
            y=traffic_column,
            color="Mobility mode",
            color_discrete_sequence=px.colors.sequential.Plasma_r,
            labels={traffic_column: variable},
            title="Mobility mode time series",
        )
    # Customize the hovers, title and yaxis
    area_plot.update_layout(hoverlabel=hovers,
    title_x=0.11,
    yaxis_title=f"{variable} ({traffic_units[variable]})")
    return area_plot

# Reset the traffic heatmap
@app.callback(
    Output("traffic-heatmap","clickData"),
    [Input("reset-button-traffic","n_clicks")])
def resetTrafficLocation(reset):
    return None

####-----------------------------------------

## Air quality

# Callback to AQ heatmap
@app.callback(
    Output("aq-heatmap", "figure"),
    Output("location-text-aq", "children"),
    Input("crossfilter-situation-aq", "value"), 
    Input("crossfilter-variable-aq", "value"),
    Input("crossfilter-timestep-range", "value"),
    Input("crossfilter-timeline-type", "value"),
    Input("aq-heatmap", "clickData"))
def drawAirQualityHeatmap(situation, variable, timestep_range, timeline_type, spatial_click_data):
    # Which dataset to use
    dataset_name=""
    if situation == "Current":
        dataset_name = "Current emissions"
    else:
        dataset_name = "Optimized emissions"
    # Filter the dataset according to inputs
    network, location, bounds = filter_dataframe(dataset_name=dataset_name,
    spatial_click_data=spatial_click_data,
    timestep_range=timestep_range,
    timeline_type=timeline_type,
    variable=variable)
    # Draw the figure
    legend_max = 1.2 * network[network[variable] != 0][variable].median()
    fig = px.density_map(data_frame=network,
    lat = "Latitude",
    lon = "Longitude",
    z = variable,
    radius=15,
    zoom=10,
    range_color = (0,legend_max),
    map_style="open-street-map",
    hover_data = {"Name": True, 
                  "Edge": False,
                  "Amount": True, 
                  "Longitude": False, 
                  "Latitude": False, 
                  variable: True},
    animation_frame="Timestep",
    title="Network heatmap"
    )
    # Customize the sliders with a larger font size and prefix
    sliders = [dict(
    currentvalue={"prefix": "Time (in minutes): "},
    # active=active_frame,
    font={"size": 14},
    pad={"t": 50}
    )]
    # Update points' opacity on click
    fig.update_layout(
        clickmode="event+select",
        sliders=sliders,
        map_bounds=bounds
    )
    # Sets the variable name and its unit as a legend to the color bar
    fig.layout["coloraxis"]["colorbar"]["title"] = f"{variable} ({aq_units[variable]})"
    # Return the figure and location
    return fig, location

# Callback to AQ bar plot
@app.callback(
    Output("aq-bar", "figure"),
    Input("crossfilter-situation-aq", "value"),
    Input("crossfilter-variable-aq", "value"),
    Input("crossfilter-timestep-range", "value"),
    Input("crossfilter-timeline-type", "value"),
    Input("aq-heatmap", "clickData"))
def drawAirQualityBar(situation, variable, timestep_range, timeline_type, spatial_click_data):
    dataset_name=""
    # Which dataset to use
    if situation == "Current":
        dataset_name = "Current emissions"
    else:
        dataset_name = "Optimized emissions"
    # Filter the dataset according to inputs
    network, location, bounds = filter_dataframe(
    dataset_name=dataset_name,
    spatial_click_data=spatial_click_data,
    timestep_range=timestep_range,
    timeline_type=timeline_type,
    variable=variable)
    # Calculate average variable value
    network = network.groupby(["Mobility mode"], observed=False).agg(
        {variable: "sum", "Amount": "sum"}
        ).reset_index()
    network["Vehicle average"] = np.round(network[variable]/network["Amount"], 4)
    # Draw the figure
    bar_plot = px.bar(data_frame=network, 
    x=variable, 
    y="Mobility mode", 
    color="Vehicle average", 
    hover_data="Amount", 
    title="Vehicle averages per mobility mode")
    # Customize the hovers, title, xaxis and legend
    bar_plot.update_layout(hoverlabel=hovers, 
    title_x=0.11, bargap=0.5, 
    xaxis_title=f"{variable} ({aq_units[variable]})", 
    yaxis_title="Mobility mode", 
    xaxis=dict(range=[0, network[variable].max()]))
    return bar_plot

# Callback to AQ stacked area chart
@app.callback(
    Output("aq-area", "figure"),
    Input("crossfilter-situation-aq", "value"),
    Input("crossfilter-variable-aq", "value"),
    Input("crossfilter-timestep-range", "value"),
    Input("crossfilter-timeline-type", "value"),
    Input("aq-heatmap", "clickData"))
def drawAirQualityArea(situation, variable, timestep_range, timeline_type, spatial_click_data):
    dataset_name = ""
    # Which dataset to use
    if situation == "Current":
        dataset_name = "Current emissions"
    else:
        dataset_name = "Optimized emissions"
    # Filter the dataset according to inputs
    network, location, bounds = filter_dataframe(
    dataset_name=dataset_name,
    spatial_click_data=spatial_click_data,
    timestep_range=timestep_range,
    timeline_type=timeline_type,
    variable=variable)
    # Calculate mobility mode specific values
    network = network.groupby(["Mobility mode", "Timestep"], observed=False).agg(
        {variable: "sum", "Amount": "sum"}
        ).reset_index()
    # Calculate average variable value
    network["Vehicle average"] =  np.round(network[variable]/ network["Amount"], 4)
    # Draw the figure
    area_plot = px.area(data_frame=network, 
    x="Timestep", 
    y=variable, 
    color="Mobility mode", 
    color_discrete_sequence=px.colors.sequential.Plasma_r, 
    hover_data=["Vehicle average", "Amount"],
    title="Mobility mode time series")
    # Customize the hovers, title and yaxis
    area_plot.update_layout(hoverlabel=hovers,
    title_x=0.11,
    yaxis_title=f"{variable} ({aq_units[variable]})")
    return area_plot

# Reset the AQ heatmap
@app.callback(
    Output("aq-heatmap","clickData"),
    [Input("reset-button-aq","n_clicks")])
def resetAirQualityLocation(reset):
    return None

####-----------------------------------------

## Livability

####-----------------------------------------

app.run(debug=True)

3602.142 Carbon monoxide
0      0.0
1      0.0
2      0.0
3    100.2
4      0.0
Name: Carbon monoxide, dtype: float64
3602.142 Carbon monoxide
0      0.0
1      0.0
2      0.0
3    100.2
4      0.0
Name: Carbon monoxide, dtype: float64
3602.142 Carbon monoxide
0      0.0
1      0.0
2      0.0
3    100.2
4      0.0
Name: Carbon monoxide, dtype: float64
3602.142 Carbon monoxide
0      0.0
1      0.0
2      0.0
3    100.2
4      0.0
Name: Carbon monoxide, dtype: float64
3602.142 Carbon monoxide
0      0.0
1      0.0
2      0.0
3    100.2
4      0.0
Name: Carbon monoxide, dtype: float64
3602.142 Carbon monoxide
0      0.0
1      0.0
2      0.0
3    100.2
4      0.0
Name: Carbon monoxide, dtype: float64
